<a href="https://colab.research.google.com/github/joaopaulolndev/deep-learning-keras-projects/blob/master/CatsAndDogs/GatoCachorro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

import numpy as np

In [90]:
# acessa o drive e lista o que tem dentro
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
test_set  training_set


In [0]:
# configurando as redes

classificador = Sequential()
classificador.add( Conv2D( 32, (3,3), input_shape=(64,64,3), activation = 'relu' ) )
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size=(2,2)))

classificador.add( Conv2D( 32, (3,3), input_shape=(64,64,3), activation = 'relu' ) )
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size=(2,2)))

classificador.add(Flatten())

classificador.add( Dense( units=128, activation='relu') )
classificador.add(Dropout(0.2))
classificador.add( Dense( units=128, activation='relu') )
classificador.add(Dropout(0.2))
classificador.add( Dense( units=1, activation='sigmoid') )

In [0]:
# compilando a rede
classificador.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
# gerando as imagens de treinamento
gerador_treinamento = ImageDataGenerator(rescale = 1./255,
                                        rotation_range = 7,
                                        horizontal_flip =True,
                                        shear_range = 0.2,
                                        height_shift_range = 0.07,
                                        zoom_range = 0.2)

In [0]:
gerador_teste = ImageDataGenerator(rescale = 1./255)

In [95]:
# recuperando a base de treinamento
base_treinamento = gerador_treinamento.flow_from_directory('/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/dataset/training_set',
                                                          target_size = (64,64),
                                                          batch_size = 32,
                                                          class_mode = 'binary')

Found 4030 images belonging to 2 classes.


In [96]:
# recuperando a base de teste
base_teste = gerador_teste.flow_from_directory('/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/dataset/test_set',
                                              target_size = (64,64),
                                              batch_size = 32,
                                              class_mode = 'binary')

Found 1052 images belonging to 2 classes.


In [0]:
# Valores de parametros para treinar bem a rede neural
#steps_per_epoch=4000/32
#print(steps_per_epoch)
#validation_steps = 1000/32
#print(validation_steps)

In [98]:
# fazendo o treinamento da base
classificador.fit_generator(base_treinamento, steps_per_epoch=100,
                           epochs = 10, validation_data = base_teste, validation_steps = 31) # no exemplo eram 5 epocas

Epoch 1/10
100/100 [==============================] - 97s 967ms/step - loss: 0.8557 - acc: 0.5709 - val_loss: 0.7178 - val_acc: 0.5685
Epoch 2/10
100/100 [==============================] - 90s 903ms/step - loss: 0.7235 - acc: 0.5888 - val_loss: 1.0934 - val_acc: 0.5536
Epoch 3/10
100/100 [==============================] - 90s 895ms/step - loss: 0.6732 - acc: 0.6313 - val_loss: 0.6670 - val_acc: 0.6083
Epoch 4/10
100/100 [==============================] - 90s 900ms/step - loss: 0.6323 - acc: 0.6610 - val_loss: 0.7037 - val_acc: 0.6407
Epoch 5/10
100/100 [==============================] - 90s 903ms/step - loss: 0.6124 - acc: 0.6791 - val_loss: 0.6872 - val_acc: 0.6174
Epoch 6/10
100/100 [==============================] - 90s 897ms/step - loss: 0.6038 - acc: 0.6796 - val_loss: 0.6146 - val_acc: 0.6670
Epoch 7/10
100/100 [==============================] - 90s 900ms/step - loss: 0.5635 - acc: 0.7066 - val_loss: 0.6024 - val_acc: 0.6781
Epoch 8/10
100/100 [==============================] - 9

# Salvando uma rede neural em um Json

In [0]:

classificador_json = classificador.to_json()
with open('/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/classificador_gato_cachorro.json','w') as json_file:
  json_file.write(classificador_json)
  
  
classificador.save_weights('/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/classificador_gato_cachorro.h5')  

# Testando uma rede neural

In [0]:
# Testando a classificacao apenas com 1 imagem
# 2.dog.jpg
imagem_teste = image.load_img('/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/dataset/test_set/gato/1.boo.jpg', target_size = (64,64))
imagem_teste = image.img_to_array(imagem_teste)
imagem_teste /= 255

In [0]:
# expande as dimensoes da imagem
imagem_teste = np.expand_dims(imagem_teste, axis = 0)

In [102]:
base_treinamento.class_indices

{'cachorro': 0, 'gato': 1}

In [103]:
classificador.predict(imagem_teste)

array([[0.03755818]], dtype=float32)

In [104]:
previsao = classificador.predict(imagem_teste)
previsao = (previsao > 0.5)

# se for menor que 0.5 a precisao eh cachorro, se for maior eh gato
if(previsao[0][0]):
  print('Gato')
else:
   print('Cachorro')

Cachorro


# Carregando uma rede neural já executada de um json

In [0]:
# carregando a rede neural salva no json
import numpy as np
from keras.models import model_from_json

In [0]:
arquivo = open('/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/classificador_gato_cachorro.json','r')

In [0]:
estrutura_rede = arquivo.read()
arquivo.close()

In [0]:
classificador = model_from_json(estrutura_rede)

In [0]:
classificador.load_weights('/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/classificador_gato_cachorro.h5')

In [0]:
# Testando a classificacao apenas com 1 imagem
# 2.dog.jpg
imagem_teste = image.load_img('/content/drive/My Drive/Colab Notebooks/DeepLearningAZ-Jones/dataset/test_set/gato/1.egipicio.jpg', target_size = (64,64))
imagem_teste = image.img_to_array(imagem_teste)
imagem_teste /= 255

In [0]:
# expande as dimensoes da imagem
imagem_teste = np.expand_dims(imagem_teste, axis = 0)

In [112]:
base_treinamento.class_indices

{'cachorro': 0, 'gato': 1}

In [113]:
classificador.predict(imagem_teste)

array([[0.58925366]], dtype=float32)

In [114]:
previsao = classificador.predict(imagem_teste)
previsao = (previsao > 0.5)

# se for menor que 0.5 a precisao eh cachorro, se for maior eh gato
if(previsao[0][0]):
  print('Gato')
else:
   print('Cachorro')

Gato
